In [1]:
!pip install tqdm

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install scipy

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
%config Completer.use_jedi = False

## Descompactação dos Dados

In [6]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [7]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [8]:
zip_object.extractall("./")

In [9]:
zip_object.close()

## Configuração de caminhos (paths)

In [10]:
dataset_path_new = "./cats_and_dogs_filtered"

In [11]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construção do modelo

### Carregando o modelo pré-treinado (MobileNetV2)

In [12]:
img_shape = (128,128,3)

In [13]:
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape, include_top=False, weights="imagenet")

In [14]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

### Congelando o modelo base

In [15]:
base_model.trainable = False

### Definição do cabeçalho personalizado da rede neural

In [16]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

#### Redução da dimensionalidade da saída do modelo base para a entrada do cabeçalho 

In [17]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [18]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [19]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

## Definição do modelo

In [20]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

## Compilar o modelo

In [22]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy",
                                                    metrics=['accuracy'])

### Geradores de Dados
Redimensionando as imagens

> Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.

Por exemplo: MobileNet (que estamos usando) suporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [23]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [24]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator = data_gen_train.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Treino do modelo

In [25]:
model.fit(train_generator, epochs=10, validation_data=valid_generator)

Epoch 1/10
16/16 [==============================] - 10s 433ms/step - loss: 0.6121 - accuracy: 0.6709 - val_loss: 0.5363 - val_accuracy: 0.7350
Epoch 2/10
16/16 [==============================] - 5s 330ms/step - loss: 0.5094 - accuracy: 0.7663 - val_loss: 0.4665 - val_accuracy: 0.8020
Epoch 3/10
16/16 [==============================] - 5s 321ms/step - loss: 0.4511 - accuracy: 0.8160 - val_loss: 0.4098 - val_accuracy: 0.8490
Epoch 4/10
16/16 [==============================] - 5s 326ms/step - loss: 0.3981 - accuracy: 0.8491 - val_loss: 0.3623 - val_accuracy: 0.8800
Epoch 5/10
16/16 [==============================] - 5s 322ms/step - loss: 0.3403 - accuracy: 0.8954 - val_loss: 0.3246 - val_accuracy: 0.8970
Epoch 6/10
16/16 [==============================] - 5s 328ms/step - loss: 0.3198 - accuracy: 0.9015 - val_loss: 0.2925 - val_accuracy: 0.9080
Epoch 7/10
16/16 [==============================] - 5s 322ms/step - loss: 0.2826 - accuracy: 0.9120 - val_loss: 0.2647 - val_accuracy: 0.9190
Epoch

## Avaliação do modelo

In [26]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 2s 214ms/step - loss: 0.2055 - accuracy: 0.9370


In [27]:
print("Loss: {:.2f}, Accuracy: {:.2f}".format(valid_loss, valid_accuracy))

Loss: 0.21, Accuracy: 0.94
